In [1]:
import numpy as np
import pandas as pd
import ast

In [2]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies=movies.merge(credits,on='title')

In [4]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [5]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [6]:
movies.dropna(inplace=True)

In [7]:
movies.duplicated().sum()

0

In [8]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [9]:
movies['genres']=movies['genres'].apply(convert)

In [10]:
movies['keywords']=movies['keywords'].apply(convert)

In [11]:
def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [12]:
movies['cast']=movies['cast'].apply(convert3)

In [13]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [14]:
movies['crew']=movies['crew'].apply(fetch_director)

In [15]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [16]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [17]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [18]:
new_df=movies[['movie_id','title','tags']]

In [19]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x ))

C:\Users\Mohammed Awan\AppData\Local\Temp\ipykernel_4632\1244981833.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x ))


In [20]:
import nltk

In [21]:
from nltk.stem.porter import PorterStemmer

In [22]:
ps=PorterStemmer()

In [23]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [24]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\Mohammed Awan\AppData\Local\Temp\ipykernel_4632\3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=5000,stop_words='english')

In [26]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [27]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
similarity=cosine_similarity(vectors)

In [30]:
similarity[0]

array([1.        , 0.69023677, 0.43619244, ..., 0.        , 0.        ,
       0.        ])

In [31]:
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [32]:
recommend('Batman Begins')

Amidst the Devil's Wings
The Punisher
Antitrust
Menace II Society
I Am Wrath


In [33]:
import pickle

In [34]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [35]:
pickle.dump(similarity,open('similarity.pkl','wb'))